# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

#### a) Make a function to tokenize the text.

In [ ]:
install.packages("tokenizers")
install.packages("stringr")
install.packages("httr")
library(tokenizers)
library(stringr)
library(httr)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘SnowballC’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
tokenize_text <- function(text) {
  # Use the tokenizer package for robust tokenization
  tokenizers::tokenize_words(text, lowercase = TRUE, strip_punct = TRUE)[[1]]
}

#### b) Make a function generate keys for ngrams.

In [ ]:
# We'll represent an (n-1)-gram context as a single string key joined by a sentinel separator.
.key_sep <- "\x1f"

key_from_ngram <- function(ngram, sep = .key_sep) {
  paste(ngram, collapse = sep)
}


#### c) Make a function to build an ngram table.

In [ ]:
# n-gram table stored as an environment: key -> named integer vector of next-word counts
build_ngram_table <- function(tokens, n, sep = .key_sep) {
  if (length(tokens) < n) return(new.env(parent = emptyenv()))
  tbl <- new.env(parent = emptyenv())

  for (i in seq_len(length(tokens) - n + 1L)) {
    context <- tokens[i:(i + n - 2L)]     # (n-1)-gram
    next_w  <- tokens[i + n - 1L]         # the nth word
    key     <- paste(context, collapse = sep)

    counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (next_w %in% names(counts)) {
      counts[[next_w]] <- counts[[next_w]] + 1L
    } else {
      counts[[next_w]] <- 1L
    }
    tbl[[key]] <- counts
  }
  attr(tbl, "n") <- as.integer(n)
  attr(tbl, "sep") <- sep
  tbl
}

#### d) Function to digest the text.

In [ ]:
digest_text <- function(text, n) {
  toks <- tokenize_text(text)
  build_ngram_table(toks, n)
}

#### e) Function to digest the url.

In [ ]:
library(httr)

digest_url <- function(url, n) {
  res <- httr::GET(url)
  stopifnot(httr::http_error(res) == FALSE)
  txt <- httr::content(res, as = "text", encoding = "UTF-8")
  digest_text(txt, n)
}

#### f) Function that gives random start.

In [ ]:
random_start <- function(tbl) {
  keys <- ls(envir = tbl, all.names = TRUE)
  if (length(keys) == 0) stop("No n-grams found — did you build the table?")
  sep <- attr(tbl, "sep", exact = TRUE)
  strsplit(sample(keys, 1), sep, fixed = TRUE)[[1]]
}

#### g) Function to predict the next word.

In [ ]:
predict_next_word <- function(tbl, context) {
  sep <- attr(tbl, "sep", exact = TRUE)
  key <- paste(context, collapse = sep)
  counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
  if (length(counts) == 0) return(NA_character_)
  sample(names(counts), size = 1, prob = as.numeric(counts))
}

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [ ]:
generate_text <- function(tbl, start = NULL, length = 20L) {
  n <- attr(tbl, "n", exact = TRUE)
  if (is.null(n)) stop("Table missing 'n' attribute — build via build_ngram_table/digest_*.")

  # If user doesn’t give start words, pick a random (n-1)-gram
  if (is.null(start) || length(start) != (n - 1L)) {
    start <- random_start(tbl)
  }

  words <- start
  for (i in seq_len(max(0L, length - length(start)))) {
    ctx <- tail(words, n - 1L)
    nx  <- predict_next_word(tbl, ctx)
    if (is.na(nx)) break
    words <- c(words, nx)
  }

  paste(words, collapse = " ")
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15.
#### ii) Using n=3, with no start word, with length=15.

In [ ]:
set.seed(2025)

url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"
tbl3 <- digest_url(url, n = 3)

# (i) Using start words "the king", length = 15
output_i <- generate_text(tbl3, start = c("the", "king"), length = 15)
cat("i) Start = 'the king' →\n", output_i, "\n\n")

# (ii) No start word, random start, length = 15
output_ii <- generate_text(tbl3, length = 15)
cat("ii) Random start →\n", output_ii, "\n")

i) Start = 'the king' →
 the king has forbidden me to marry another husband am not i shall ride upon 

ii) Random start →
 song was over the lake and herself into her little daughter’s hand and was about 


#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15.
#### ii) Using n=3, with no start word, with length=15.

In [ ]:
url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"
tbl3 <- digest_url(url, n = 3)

# (i) Using start words "the king", length = 15
output_i <- generate_text(tbl3, start = c("the", "king"), length = 15)
cat("i) Start = 'the king' →\n", output_i, "\n\n")

# (ii) No start word, random start, length = 15
output_ii <- generate_text(tbl3, length = 15)
cat("ii) Random start →\n", output_ii, "\n")

i) Start = 'the king' →
 the king is also defective you may demand a refund of any kind express or 

ii) Random start →
 mine his vigil of arms already cited and in token of the best illustration contributed 


#### c) Explain in 1-2 sentences the difference in content generated from each source.

When you specify "the king" as the starting words, the model produces coherent fairy-tale continuations involving royal characters. With a random start, it begins from an arbitrary place in the corpus, so the generated text is more varied and sometimes less structured.

## Question 3
#### a) What is a language learning model?

A language learning model is a probability distribution over words. You put some input into the probability distribution, which is conditioning the probability distribution, and you get an output fom the probability distribution. It basically predicts the next word (token) given the previous words.


#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

1. The raw text is split into tokens (usually words).
2. The model uses previous tokens to form a probability distribution of possible next words.
3. The model selects the next word based on these probabilities.
4. The selected word is appended to the text and the process repeats to generate a full sentence.

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | A program which lets you interact with all the functionality of a system (the operating system, in this context, though other systems have shells too); called a "shell" because its a shell around the O. |
| **Terminal emulator** | A shell is something that sits between a user and an OS and the "place" the shell sits is the terminal emulator (HOSTS a shell). A terminal emulator can do special stuff with bytes returned by the shell to present effects, control, etc. |
| **Process** | Something running on your computer |
| **Signal** | Things we can send to processes to tell them to do something |
| **Standard input** | Each process has these; they can read characters from the input and write to the output. |
| **Standard output** | Each process has these; they can read characters from the input and write to the output. |
| **Command line argument** | Things we pass to a process when we start it. |
| **The environment** | All the stuff a process can see when its running; ok to imagine that when one process starts another, the child process sees all the same stuff as the parent. |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?

The programs are fine, xargs, and grep.

#### b) Explain what this command is doing, part by part.

find . -iname "*.R" : searches the current directory (.) and subdirectories for files whose names end in .R, case-sensitive.

xargs grep read_csv : Runs grep read_csv on each of those files, searching inside them for the string "read_csv"

Overall the command finds all R files in the project and prints the lines where read_csv is used.

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions.
#### a) Show the response when you run `docker run hello-world`.

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/

#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.

talfayer@tals-MacBook-Pro ~ % docker run -e PASSWORD=mypassword -p 8787:8787 -v "$PWD":/home/rstudio -d rocker/rstudio

Unable to find image 'rocker/rstudio:latest' locally
latest: Pulling from rocker/rstudio
b8a35db46e38: Pull complete
2c9ba66d5dbe: Pull complete
2034506aa72f: Pull complete
bcce866b1806: Pull complete
91ed5b86de88: Pull complete
cc9c938c1f51: Pull complete
5d246ec925db: Pull complete
08e74fd5985d: Pull complete
39038e16d1ba: Pull complete
664fb1818bbb: Pull complete
191985778909: Pull complete
9c1a4a0706b7: Pull complete
e2804bef35e8: Pull complete
5b219f62ce36: Pull complete
abd0190d83fb: Pull complete
bc9245ceaac5: Pull complete
a730ff463d58: Pull complete
3665120d345d: Pull complete
Digest: sha256:9f85211a666fb426081a6f5a01f9f9f51655262258419fa21e0ce38a5afc78d8
Status: Downloaded newer image for rocker/rstudio:latest
c82b406f84fdb79deae2e74e26efc2255a2822cf3d9cae00387c3fe966e455b8

#### c) How do you log in to the RStudio server?

using the username rstudio and the password you made in the docker run command.